In [ ]:
# !pip install -U datasets
# !pip install fsspec==2023.9.2
# !pip install transformers datasets scikit-learn tqdm
# !pip install torchmetrics


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, hamming_loss
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
from torch.cuda.amp import autocast, GradScaler
import torchmetrics

c:\Users\ahmad\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 1. Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
from datasets import load_dataset

# 1) Load full train split
ds_full = load_dataset("rntc/mimic-icd-visit", split="train")

# 2) Subset first 50k examples
ds_small = ds_full.select(range(50000))  # returns a Dataset object :contentReference[oaicite:5]{index=5}

# 3) Convert to pandas
df = ds_small.to_pandas()


In [ ]:
# 3) Train/validation split
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)


In [ ]:

# 4) Tokenizer
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")


In [ ]:
# 5. Encode labels
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(train_df['icd_code'])
y_val   = mlb.transform(val_df['icd_code'])

c:\Users\ahmad\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\preprocessing\_label.py:909: UserWarning: unknown class(es) ['A240', 'A5619', 'B1711', 'B188', 'B3323', 'B376', 'B3783', 'B463', 'B949', 'C028', 'C44229', 'C716', 'C7650', 'C7A026', 'C7A029', 'C8350', 'C8594', 'C9221', 'C9242', 'D0581', 'D1610', 'D169', 'D210', 'D224', 'D334', 'D3611', 'D376', 'D4622', 'D4989', 'E034', 'E0921', 'E09319', 'E0942', 'E103291', 'E103511', 'E113213', 'E11331', 'E11620', 'E229', 'E233', 'E7029', 'E70319', 'E7881', 'E838', 'F10282', 'F11929', 'F13280', 'F13929', 'F19150', 'F19151', 'F1917', 'F19181', 'F19232', 'F980', 'G1222', 'G44029', 'G4763', 'G547', 'G5611', 'G5711', 'G5712', 'G5773', 'H02209', 'H02529', 'H0263', 'H0266', 'H059', 'H10419', 'H25043', 'H26222', 'H33319', 'H35041', 'H353130', 'H35341', 'H35349', 'H4011X2', 'H4031X0', 'H5015', 'H5021', 'H5040', 'H59091', 'H6022', 'H608X3', 'H6592', 'H66012', 'H70092', 'H8111', 'H906', 'I011', 'I072', 'I4902', 'I69132', 'I69264', 

In [ ]:
# 6. Dataset class
class ICD9Dataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])
        label = self.labels[idx]
        enc = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids':      enc['input_ids'].squeeze(0),
            'attention_mask': enc['attention_mask'].squeeze(0),
            'labels':         torch.tensor(label, dtype=torch.float)
        }

In [ ]:
# 7. Datasets & Dataloaders
train_ds = ICD9Dataset(train_df['cleaned_text'], y_train, tokenizer)
val_ds   = ICD9Dataset(val_df['cleaned_text'],   y_val,   tokenizer)

batch_size = 16
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=0)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=0)

In [ ]:
# 8. Model definition
class ClinicalBERTClassifier(nn.Module):
    def __init__(self, num_labels):
        super().__init__()
        self.bert = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(768, num_labels)
        for name, p in self.bert.named_parameters():
            if not ("encoder.layer.11" in name or "pooler" in name):
                p.requires_grad = False

    def forward(self, input_ids, attention_mask):
        out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = out.pooler_output
        return self.classifier(self.dropout(pooled))

model = ClinicalBERTClassifier(num_labels=len(mlb.classes_)).to(device)

In [ ]:
# 9. Optimizer, loss, scheduler, scaler
differential_params = [
    {'params': model.classifier.parameters(), 'lr': 1e-3},
    {'params': model.bert.pooler.parameters(),  'lr': 5e-4},
    {'params': model.bert.encoder.layer[11].parameters(), 'lr': 2e-5},
]
optimizer = torch.optim.AdamW(differential_params)
loss_fn = nn.BCEWithLogitsLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=1)
scaler = GradScaler()

C:\Users\ahmad\AppData\Local\Temp\ipykernel_7756\3316921300.py:10: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [ ]:
# # 10. TorchMetrics setup
# num_labels = len(mlb.classes_)
# metric_args = {'multilabel': True, 'num_labels': num_labels, 'threshold': 0.5, 'compute_on_step': False}

In [ ]:
# 10. TorchMetrics setup
num_labels = len(mlb.classes_)

train_acc    = torchmetrics.Accuracy(task='multilabel', num_labels=num_labels, threshold=0.3).to(device)
train_f1     = torchmetrics.F1Score(task='multilabel', num_labels=num_labels, threshold=0.3).to(device)
train_prec   = torchmetrics.Precision(task='multilabel', num_labels=num_labels, threshold=0.3).to(device)
train_recall = torchmetrics.Recall(task='multilabel', num_labels=num_labels, threshold=0.3).to(device)
train_ham = torchmetrics.HammingDistance(task='multilabel', num_labels=num_labels).to(device)

val_acc      = torchmetrics.Accuracy(task='multilabel', num_labels=num_labels, threshold=0.3).to(device)
val_f1       = torchmetrics.F1Score(task='multilabel', num_labels=num_labels, threshold=0.3).to(device)
val_prec     = torchmetrics.Precision(task='multilabel', num_labels=num_labels, threshold=0.3).to(device)
val_recall   = torchmetrics.Recall(task='multilabel', num_labels=num_labels, threshold=0.3).to(device)
val_ham   = torchmetrics.HammingDistance(task='multilabel', num_labels=num_labels).to(device)


In [ ]:
# 11. EarlyStopping class
class EarlyStopping:
    def __init__(self, patience=2, delta=0, path='best_model.pt'):
        self.patience = patience
        self.delta = delta
        self.path = path
        self.counter = 0
        self.best_score = None
        self.best_loss = np.inf
        self.early_stop = False

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None or score > self.best_score + self.delta:
            self.best_score = score
            self.best_loss = val_loss
            torch.save(model.state_dict(), self.path)
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# early_stopping = EarlyStopping(patience=2, path='best_model.pt')

In [ ]:
# 12. Train & Eval functions
def train_epoch():
    model.train()
    train_acc.reset(); train_f1.reset()
    train_prec.reset(); train_recall.reset(); train_ham.reset()
    total_loss = 0.0

    for batch in tqdm(train_loader, desc="Training"):
        ids    = batch['input_ids'].to(device)
        mask   = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        with autocast():
            logits = model(ids, mask)
            loss   = loss_fn(logits, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()

        preds = torch.sigmoid(logits)
        train_acc.update(preds, labels)
        train_f1.update(preds, labels)
        train_prec.update(preds, labels)
        train_recall.update(preds, labels)
        train_ham.update(preds, labels)

    return {
        'loss': total_loss / len(train_loader),
        'acc': train_acc.compute().item(),
        'f1': train_f1.compute().item(),
        'precision': train_prec.compute().item(),
        'recall': train_recall.compute().item(),
        'hamming_loss': train_ham.compute().item() / (batch_size * num_labels)
    }


def eval_epoch():
    model.eval()
    val_acc.reset(); val_f1.reset()
    val_prec.reset(); val_recall.reset(); val_ham.reset()
    total_loss = 0.0

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            ids    = batch['input_ids'].to(device)
            mask   = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            with autocast():
                logits = model(ids, mask)
                loss   = loss_fn(logits, labels)
            total_loss += loss.item()

            preds = torch.sigmoid(logits)
            val_acc.update(preds, labels)
            val_f1.update(preds, labels)
            val_prec.update(preds, labels)
            val_recall.update(preds, labels)
            val_ham.update(preds, labels)

    return {
        'loss': total_loss / len(val_loader),
        'acc': val_acc.compute().item(),
        'f1': val_f1.compute().item(),
        'precision': val_prec.compute().item(),
        'recall': val_recall.compute().item(),
        'hamming_loss': val_ham.compute().item() / (batch_size * num_labels)
    }

In [ ]:
# 13. Training loop
epochs = 15
for epoch in range(1, epochs+1):
    print(f"\nEpoch {epoch}/{epochs}")
    train_metrics = train_epoch()
    val_metrics   = eval_epoch()
    scheduler.step(val_metrics['loss'])

    print(f"Train — loss: {train_metrics['loss']:.4f}, acc: {train_metrics['acc']:.4f}, f1: {train_metrics['f1']:.4f}, ham: {train_metrics['hamming_loss']:.4f}")
    print(f"Val   — loss: {val_metrics['loss']:.4f}, acc: {val_metrics['acc']:.4f}, f1: {val_metrics['f1']:.4f}, ham: {val_metrics['hamming_loss']:.4f}")

    # early_stopping(val_metrics['loss'], model)
    # if early_stopping.early_stop:
    #     print("Early stopping triggered.")
    #     break



Epoch 1/15


Training:   0%|          | 0/2813 [00:00<?, ?it/s]C:\Users\ahmad\AppData\Local\Temp\ipykernel_7756\1835299561.py:14: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Validating:   0%|          | 0/313 [00:00<?, ?it/s]C:\Users\ahmad\AppData\Local\Temp\ipykernel_7756\1835299561.py:51: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Validating: 100%|██████████| 313/313 [00:52<00:00,  5.98it/s]


Train — loss: 0.0070, acc: 0.9977, f1: 0.0813, ham: 0.0000
Val   — loss: 0.0055, acc: 0.9987, f1: 0.1755, ham: 0.0000

Epoch 2/15


Validating: 100%|██████████| 313/313 [00:51<00:00,  6.09it/s]


Train — loss: 0.0055, acc: 0.9987, f1: 0.1916, ham: 0.0000
Val   — loss: 0.0053, acc: 0.9987, f1: 0.2182, ham: 0.0000

Epoch 3/15


...

Train — loss: 0.0044, acc: 0.9988, f1: 0.2971, ham: 0.0000
Val   — loss: 0.0050, acc: 0.9987, f1: 0.2815, ham: 0.0000

Epoch 14/15


Validating: 100%|██████████| 313/313 [00:40<00:00,  7.75it/s]


Train — loss: 0.0044, acc: 0.9988, f1: 0.2968, ham: 0.0000
Val   — loss: 0.0050, acc: 0.9987, f1: 0.2823, ham: 0.0000

Epoch 15/15


Validating: 100%|██████████| 313/313 [00:40<00:00,  7.65it/s]

Train — loss: 0.0044, acc: 0.9988, f1: 0.2981, ham: 0.0000
Val   — loss: 0.0050, acc: 0.9987, f1: 0.2824, ham: 0.0000


In [ ]:
torch.save(model.state_dict(), 'best_model.pt')


In [ ]:
# 14. Load best model
model.load_state_dict(torch.load('best_model.pt'))

<All keys matched successfully>

In [ ]:
# 15. Inference function
def predict(text, top_k=5, threshold=0.5):
    model.eval()
    enc = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt'
    )
    ids, mask = enc['input_ids'].to(device), enc['attention_mask'].to(device)
    with torch.no_grad(), autocast():
        logits = model(ids, mask)
    probs = torch.sigmoid(logits).cpu().numpy()[0]
    above = [mlb.classes_[i] for i in np.where(probs>=threshold)[0]]
    topk  = [mlb.classes_[i] for i in probs.argsort()[-top_k:][::-1]]
    return {'above_threshold': above, 'top_k': topk}


In [ ]:
# Example inference
sample = "Allergies: shellfish derived Chief Complaint: Right-sided weakness Major Surgical or Invasive Procedure: Lumbar puncture History of Present Illness: Mr. [MASKED] is a [MASKED] man with a past medical history of thyroid cancer status post resection followed by [MASKED], hyperlipidemia who presents with 10 days of discrete episodes of headache, lightheadedness, and focal neurologic deficits. History is obtained by wife at the bedside as patient not able to participate. On [MASKED], wife notes that patient had sudden onset massive headache with associated lightheadedness and tunnel vision. He also experienced numbness in his right arm. There was no vertigo. It is unclear how long this episode lasted but patient went to bed and slept for approximately 15 hours. On awakening the following day, patient felt back to baseline. He called his primary care physician and went in for evaluation that day. His PCP performed [MASKED] lightheadedness workup and found no neurologic deficits. He felt his symptoms were most likely secondary to migraine headache. Two days later, on [MASKED], patient sent wife incoherent text message via phone. She called him afterward and he had difficulty getting the words out when speaking with her. The wife took patient to [MASKED] where he had a noncontrast [MASKED] CT which was unremarkable. He would that he was diagnosed with migraine and sent home. The whole episode lasted approximately 1 hour. Patient and his wife followed up with a neurologist through [MASKED] the following day and he had an MRI brain without contrast/MRA [MASKED] and neck as an outpatient. Imaging was unremarkable. There are no notes to review from this encounter, but the wife reports the neurologist agreed the etiology of his symptoms was most likely migraine. The following day, patient complained of headache which felt like a headband putting pressure on his whole [MASKED], right worse than left side. The wife began to document his symptoms and she notes: 12 [MASKED] lightheadedness and vision issues 12:15 [MASKED] went to bed and slept until 2 [MASKED] 2:05 [MASKED] woke up with a raging headache behind eyes and photophobia. She documents an exaggerated startle, jumping every time she walks by her opens a soda can. He went to sleep shortly afterward and awoke 4 hours later. 6:15 [MASKED] awoke with continued lightheadedness and headache 9 [MASKED] out of bed, feeling better He slept until 8:30 AM. He was asymptomatic the following day. Today at 7 [MASKED] he experienced sudden onset right hand numbness which traveled to his right arm and leg. He also had twitching of his right face and clenched his right eye in pain. He also had difficulty speaking, getting any words out. EMS was called and he was brought to [MASKED] where code stroke was called. Noncontrast [MASKED] CT was negative for bleed or other intracranial abnormality. Shortly after arrival to the ED his symptoms resolved and TPA was deferred. He began to complain of a headache at this time. He was transferred to [MASKED] for further management. Neurology was consulted for question of seizures versus atypical migraine. Unable to complete review of systems, though positive neurologic review of systems are noted as above. Specifically, patient's wife denies a history of headache or migraines. She denies any recent fever or chills. Past Medical History: Thyroid cancer status post resection. No chemo or radiation. Diagnosed in [MASKED], managed at [MASKED]. On maintenance levothyroxine. Hyperlipidemia Social History: [MASKED] Family History: No family history of stroke, seizure, or migraine. Physical Exam: ============== ADMISSION EXAM ============== Vitals: Tm: 102.5 P: 40-70 r: 16 BP: 136/90 SaO2: 99 on room air Blood glucose 89 General: Agitated, thrashing in bed, pulling off telemetry leads, and trying to get out of bed, holding [MASKED] periodically and moaning HEENT: NC/AT, positive photophobia, no scleral icterus noted, MMM, no lesions noted in oropharynx, incessant yawning Neck: Supple, No nuchal rigidity Pulmonary: Normal work of breathing Cardiac: Bradycardic, regular rhythm, warm, well-perfused Abdomen: soft, non-distended Extremities: No [MASKED] edema. He repeatedly lifts his left arm above his [MASKED] and then yawns and almost stereotyped fashion multiple times throughout the examination. Skin: no rashes or lesions noted. Neurologic: -Mental Status: Agitated, moving back and forth in bed, appears uncomfortable, grimacing and moaning. Will attend to examiner briefly when shouting his name but then turns away and repositions himself. Does not track. When asked his name, he replies okay. He perseverates on I am sorry. He attempts to answer the question, but there is never a straight response. At times, he repeats what? over and over. He then appears to get agitated and says leave me alone. Other spontaneous speech includes oh God. It is difficult to say if he follows any commands. His speech does not sound dysarthric. He is unable to participate in any further language testing. -Cranial Nerves: He is severely photophobic so pupil examination is limited. Overall pupils appear symmetric and reactive. He moves eyes in both horizontal directions. He has bilateral blink to threat. Unable to visualize fundi bilaterally. No facial droop, facial musculature symmetric. Palate elevates symmetrically. Tongue protrudes in midline. -Motor: Normal bulk, tone throughout. No adventitious movements, such as tremor, noted. Arms and legs are antigravity and appear strong, though formal confrontational testing is not possible. -Sensory: He withdraws to noxious stimuli in all 4 extremities. -DTRs: Bi Tri [MASKED] Pat Ach L 2 2 2 2 2 R 2 2 2 2 2 Plantar response was flexor bilaterally. -Coordination: Grabs for his IV with his left hand and pulls off telemetry leads with his right hand. -Gait: Able to bring himself to a standing position unassisted. DISCHARGE EXAM: Nonfocal Pertinent Results: ==== LABS ==== - CBC [MASKED] 10:00PM BLOOD WBC-6.7 RBC-4.63 Hgb-14.0 Hct-42.4 MCV-92 MCH-30.2 MCHC-33.0 RDW-12.6 RDWSD-42.2 Plt [MASKED] [MASKED] 10:00PM BLOOD Neuts-57.0 [MASKED] Monos-8.8 Eos-2.1 Baso-0.3 Im [MASKED] AbsNeut-3.83 AbsLymp-2.13 AbsMono-0.59 AbsEos-0.14 AbsBaso-0.02 [MASKED] 05:59AM BLOOD WBC-7.8 RBC-4.43* Hgb-13.3* Hct-40.4 MCV-91 MCH-30.0 MCHC-32.9 RDW-12.4 RDWSD-41.8 Plt [MASKED] [MASKED] 05:25AM BLOOD WBC-4.8 RBC-4.46* Hgb-13.6* Hct-41.8 MCV-94 MCH-30.5 MCHC-32.5 RDW-12.7 RDWSD-43.9 Plt [MASKED] [MASKED] 05:25AM BLOOD Neuts-53.2 [MASKED] Monos-10.9 Eos-2.7 Baso-0.4 Im [MASKED] AbsNeut-2.52 AbsLymp-1.55 AbsMono-0.52 AbsEos-0.13 AbsBaso-0.02 - Coag [MASKED] 10:23PM BLOOD [MASKED] PTT-27.2 [MASKED] [MASKED] 05:59AM BLOOD [MASKED] PTT-26.5 [MASKED] - CMP [MASKED] 10:00PM BLOOD Glucose-98 UreaN-19 Creat-1.2 Na-140 K-3.7 Cl-100 HCO3-28 AnGap-16 [MASKED] 10:00PM BLOOD ALT-15 AST-16 CK(CPK)-56 AlkPhos-25* TotBili-0.2 [MASKED] 05:59AM BLOOD Glucose-126* UreaN-17 Creat-1.1 Na-139 K-3.9 Cl-103 HCO3-22 AnGap-18 [MASKED] 05:25AM BLOOD ALT-13 AST-17 AlkPhos-22* TotBili-0.3 [MASKED] 05:25AM BLOOD Glucose-125* UreaN-9 Creat-1.0 Na-141 K-4.2 Cl-103 HCO3-29 AnGap-13 [MASKED] 10:00PM BLOOD Albumin-4.3 Calcium-9.1 Phos-3.8 Mg-2.0 [MASKED] 05:59AM BLOOD Calcium-8.6 Phos-3.2 Mg-1.8 [MASKED] 05:25AM BLOOD Calcium-8.6 Phos-3.8 Mg-2.1 - Other [MASKED] 10:00PM BLOOD cTropnT-<0.01 [MASKED] 05:59AM BLOOD TSH-0.37 [MASKED] 10:00PM BLOOD ASA-NEG Ethanol-NEG Acetmnp-NEG Bnzodzp-NEG Barbitr-NEG Tricycl-NEG [MASKED] 10:14PM BLOOD Lactate-1.0 - Urine [MASKED] 02:47AM URINE Blood-TR Nitrite-NEG Protein-TR Glucose-NEG Ketone-10 Bilirub-NEG Urobiln-NEG pH-6.5 Leuks-NEG [MASKED] 02:47AM URINE Color-Straw Appear-Clear Sp [MASKED] [MASKED] 02:47AM URINE RBC-3* WBC-1 Bacteri-NONE Yeast-NONE Epi-0 [MASKED] 02:47AM URINE bnzodzp-NEG barbitr-NEG opiates-NEG cocaine-NEG amphetm-NEG oxycodn-NEG mthdone-NEG - CSF [MASKED] 12:43AM CEREBROSPINAL FLUID (CSF) WBC-270 RBC-0 Polys-0 [MASKED] [MASKED] 12:43AM CEREBROSPINAL FLUID (CSF) WBC-244 RBC-0 Polys-0 [MASKED] Monos-4 Other-2 [MASKED] 12:43AM CEREBROSPINAL FLUID (CSF) TotProt-211* Glucose-63 [MASKED] 01:00PM CEREBROSPINAL FLUID (CSF) WBC-230 RBC-1* Polys-0 [MASKED] Monos-3 Other-3 [MASKED] 09:42AM CEREBROSPINAL FLUID (CSF) TotProt-212* Glucose-58 LD(LDH)-23 [MASKED] 09:42AM CEREBROSPINAL FLUID (CSF) HIV1 VL-NOT DETECT - Micro [MASKED] CSF;SPINAL FLUID GRAM STAIN-FINAL; FLUID CULTURE-PRELIMINARY GRAM STAIN (Final [MASKED]: NO POLYMORPHONUCLEAR LEUKOCYTES SEEN. NO MICROORGANISMS SEEN. This is a concentrated smear made by cytospin method, please refer to hematology for a quantitative white blood cell count.. FLUID CULTURE (Preliminary): [MASKED] CSF;SPINAL FLUID GRAM STAIN-FINAL; FLUID CULTURE-PRELIMINARY GRAM STAIN (Final [MASKED]: NO POLYMORPHONUCLEAR LEUKOCYTES SEEN. NO MICROORGANISMS SEEN. This is a concentrated smear made by cytospin method, please refer to hematology for a quantitative white blood cell count. FLUID CULTURE (Preliminary): [MASKED] CSF VDRL, VZV, CMV, EBB, Autoimmune Encephalopathy Panel, Paraneoplastic Panel, Enterovirus pending [MASKED] Blood (LYME) Lyme IgG-PENDING; Lyme IgM-PENDING [MASKED] BLOOD CULTURE Blood Culture, Routine-PENDING [MASKED] BLOOD CULTURE Blood Culture, Routine-PENDING [MASKED] URINE URINE CULTURE-PENDING [MASKED] 12:43AM CEREBROSPINAL FLUID (CSF) HERPES SIMPLEX VIRUS PCR-negative [MASKED] 05:10AM BLOOD WBC-5.6 RBC-4.46* Hgb-13.4* Hct-41.5 MCV-93 MCH-30.0 MCHC-32.3 RDW-12.7 RDWSD-43.0 Plt [MASKED] [MASKED] 05:10AM BLOOD [MASKED] PTT-25.8 [MASKED] [MASKED] 05:10AM BLOOD Glucose-94 UreaN-11 Creat-1.0 Na-140 K-3.8 Cl-100 HCO3-26 AnGap-18 [MASKED] 05:25AM BLOOD ALT-13 AST-17 AlkPhos-22* TotBili-0.3 [MASKED] 05:10AM BLOOD Calcium-8.8 Phos-4.7* Mg-2.0 [MASKED] 05:59AM BLOOD TSH-0.37 [MASKED] 05:10AM BLOOD HIV Ab-Negative ========= PATHOLOGY ========= [MASKED] CSF Cytology-negative for malignant cells [MASKED]- CSF Cytology pending ======= IMAGING ======= - [MASKED] MR [MASKED] CONTRAST 1. Study is moderately degraded by motion. 2. No evidence of venous sinus thrombosis. 3. Within limits of study, no definite leptomeningeal enhancement identified. 4. New nonspecific diffuse subarachnoid signal abnormality, as described. While finding is compatible with meningitis, differential considerations include subarachnoid hemorrhage, leptomeningeal carcinomatosis, sequelae of hyperoxygenation therapy or artifact. If clinically indicated, noncontrast [MASKED] CT may be obtained to evaluate for presence of new subarachnoid hemorrhage since [MASKED] noncontrast [MASKED] CT prior exam. 5. Paranasal sinus disease as described. - [MASKED] EEG This is an abnormal continuous ICU monitoring study because of continuous slowing on the left, maximal temporally, indicative of focal dysfunction. No pushbutton activations, epileptiform discharges, or seizures are recorded. Compared to the prior day's recording, there are no significant changes. Of note, sinus bradycardia to the [MASKED] is seen during portions of the record. Brief Hospital Course: Mr. [MASKED] is a [MASKED] year old man with history of thyroid cancer (s/p resection in [MASKED] and hyperlipidemia who presented with a ten day course of episodic headaches, lightheadedness and focal neurological deficits (extremity numbness, aphasia). He was found to have CSF pleomorphic lymphocytosis, and nonspecific diffuse subarachnoid signal abnormality on MRI. He was given a provisional diagnosis of HaNDL syndrome, pending additional tests and CSF studies, and was discharged on Verapamil 120mg for prophylactic headache treatment. He will follow-up with outpatient neurologist. #Neuro On presentation at [MASKED], the patient was acutely agitated and confused, with a low grade fever and sinus bradycardia. He had a severe headache, multiple neurological deficits and exhibited stereotyped movements. [MASKED] at outside hospital revealed no acute intracranial process. Patient was admitted to the ICU for EEG and neurological monitoring, out of concern for meningoencephalitis vs seizure vs neoplastic process. LP ([MASKED]) revealed elevated opening pressure, lymphocytic pleiocytosis, elevated protein level, and no malignant cells. The patient was initially maintained on droplet precautions, and treated empirically for meningitis with ceftriaxone, vancomycin and acyclovir until HSV PCR was negative and CSF cultures showed no bacterial growth. His symptoms had largely resolved the morning after admission and he was subsequently transferred to the floor. His symptoms did not reoccur throughout the rest of his hospital stay. EEG showed continuous slowing on the left, maximal temporally, indicative of focal dysfunction, with no epileptiform discharges. MRI revealed new nonspecific diffuse subarachnoid FLAIR hyperintensities, but no evidence of venous sinus thrombosis or definite leptomeningeal enhancement. An additional [MASKED] LP was obtained on [MASKED] for further CSF studies, with results showing continued lymphocytic pleocytosis w/ elevated protein level but improved from previous tap. At the time of discharge, the patient most closely fit the criteria for syndrome of transient Headache and Neurological Deficits with cerebrospinal fluid Lymphocytosis (HaNDL). However, HaNDL remains a provisional diagnosis until all pending results are back and other etiologies are excluded. If HaNDL is deemed to be the final diagnosis, it should be noted that it is usually a self-limiting disease and only prophylactic therapy for associated headaches is recommended. The patient was thus started on verapamil 120mg for symptomatic improvement and headache prophylaxis. #Cardiovascular CXR showed no acute cardiopulmonary process. The patient was monitored on telemetry with no evidence of atrial fibrillation. Atorvastatin 40mg daily was continued per home regimen. Transition Issues: -Pt will need to continue taking Verapamil SR 120mg daily in the near future as prophylactic therapy for migrainous headaches -Pt will need to follow up with Neurology as scheduled -Pt instructed to present to ED for evaluation if develops recurrent neurologic deficits -Pt to be informed if abnormal CSF studies present Medications on Admission: The Preadmission Medication list is accurate and complete. 1. Levothyroxine Sodium 150 mcg PO DAILY 2. Atorvastatin 40 mg PO QPM 3. ValACYclovir 2 g PO BID:PRN Cold sore Discharge Medications: 1. Verapamil SR 120 mg PO Q24H RX *verapamil 120 mg 1 tablet(s) by mouth Daily Disp #*30 Tablet Refills:*2 2. Atorvastatin 40 mg PO QPM 3. Levothyroxine Sodium 150 mcg PO DAILY 4. ValACYclovir 2 g PO BID:PRN Cold sore Discharge Disposition: Home Discharge Diagnosis: Migraine w/ focal neurologic deficits and lymphocytic pleocytosis Discharge Condition: Mental Status: Clear and coherent. Level of Consciousness: Alert and interactive. Activity Status: Ambulatory - Independent. Discharge Instructions: Dear Mr. [MASKED], You were hospitalized at [MASKED] due to recurrent migraines associated w/ focal neurologic deficits. You were initially in the NeuroICU and later transferred to the general floor. You underwent extensive laboratory workup indicative of inflammation in your cerebrospinal fluid. You were empirically treated with antibiotics and antiviral agents which were stopped when subsequent cultures were negative. You underwent EEG which showed no sign of seizure activity. Due to appearing clinically stable on exam and w/ no recurrent symptoms, you will be discharged from the hospital. Please continue taking Verapamil SR 120mg daily at this time. Please continue your other home medications. Please follow up with your PCP and [MASKED] as listed below. If you find yourself having acute focal neurologic deficits in the future, please go to your nearest ED for evaluation. It was a pleasure taking care of you, [MASKED] Neurology Team Followup Instructions: [MASKED]"
print(predict(sample))




{'above_threshold': ['E785', 'I10'], 'top_k': ['E785', 'I10', 'Z85850', 'E039', 'Z87891']}


C:\Users\ahmad\AppData\Local\Temp\ipykernel_7756\2150731214.py:12: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), autocast():
